In [33]:
import numpy as np
import pdb
from PIL import Image
from scipy import ndimage
import os
import unittest
from scipy.ndimage import maximum_filter
from numpy.lib.stride_tricks import as_strided

In [34]:
png_file = Image.open("C:\\Data\\2.png")
array_image = np.array(png_file)
kernel_3d = np.random.rand(3, 3, 4)
kernel_32d = np.random.rand(3, 3, 32)
png_file_1 = Image.open("C:\\Data\\1.png")
array_image_1 = np.array(png_file_1)
png_file_2 = Image.open("C:\\Data\\7.png")
array_image_2 = np.array(png_file_2)

In [35]:
np.random.rand(2, 2)

array([[0.68194765, 0.9387682 ],
       [0.20679594, 0.11044555]])

In [36]:
array_image_1.shape

(343, 345, 4)

In [37]:
test = np.random.rand(294, 292, 32)

In [38]:
to_stack = []

In [39]:
kernel_3d

array([[[0.73397861, 0.64129455, 0.06803645, 0.34551223],
        [0.43150178, 0.90597488, 0.78971113, 0.86288967],
        [0.02461113, 0.48845781, 0.52370474, 0.36471879]],

       [[0.2144196 , 0.3743208 , 0.68579489, 0.69816618],
        [0.19957252, 0.92960974, 0.30832836, 0.12292809],
        [0.57540116, 0.97282522, 0.77090232, 0.66802682]],

       [[0.39748781, 0.96250743, 0.72480712, 0.50615882],
        [0.53190362, 0.15114817, 0.62348178, 0.46887625],
        [0.90264513, 0.26792892, 0.12351407, 0.22916688]]])

In [40]:
def reshape_dimensions_convolution(input_height, input_width, filter_height, filter_width, stride):
    output_height = int(((input_height - filter_height) / stride) + 1)
    output_width  = int(((input_width - filter_width)  / stride) + 1)
    return (output_height, output_width)

In [41]:
reshape_dimensions_convolution(3, 3, 2, 2, 1)

(2, 2)

In [42]:
def reshape_dimensions_pooling(input_height, input_width, f, s):
    output_height = int(((input_height - f) / s) + 1)
    output_width = int(((input_width - f) / s) + 1)
    return (output_height, output_width)

In [43]:
def forward_pass(image):
    shape = image.shape
    kernels = create_kernels(32, 4)
    kernels2 =  create_kernels(64, 32)
    to_stack_values = []
    dimensions = reshape_dimensions_convolution(shape[0], shape[1], 3, 3, 1) 
    for i in range(len(kernels)):
        to_stack_values.append(np.squeeze(convolve_one_step(image, kernels[i]).reshape(dimensions[0], dimensions[1], 1)))
    stacked_filters = np.stack(to_stack_values, axis = 2)
    first_ReLU = ReLUv2(stacked_filters)
    pool_to_reshape = reshape_dimensions_pooling(first_ReLU.shape[0], first_ReLU.shape[1], 2, 2)
    first_max_pool = np.array(max_pooling(first_ReLU)).reshape(pool_to_reshape[0], pool_to_reshape[1], 32)
    to_stack_values.clear()
    shape2 = first_max_pool.shape
    dimensions2 = reshape_dimensions_convolution(shape2[0], shape2[1], 3, 3, 1)
    for f in range(len(kernels2)):
        to_stack_values.append(np.squeeze(convolve_one_step(first_max_pool, kernels2[f]).reshape(dimensions2[0], dimensions2[1], 1)))
    stacked_filters2 = np.stack(to_stack_values, axis = 2)
    second_ReLU = np.array(ReLUv2(stacked_filters2))
    pool_to_reshape2 = reshape_dimensions_pooling(second_ReLU.shape[0], second_ReLU.shape[1], 2, 2)
    second_max_pool = np.array(max_pooling(second_ReLU).reshape(pool_to_reshape[2], pool_to_reshape[2]), 64)
    return second_max_pool
    

In [44]:
#forward_pass(array_image)

In [45]:
shajp = test.shape

In [46]:
to_reshape = reshape_dimensions_convolution(shajp[0], shajp[1], 3, 3, 1)

In [47]:
alfa.shape

NameError: name 'alfa' is not defined

In [48]:
def create_kernels(number_to_create, depth):
    every_kernel = []
    for f_u in range(number_to_create):
        every_kernel.append(np.random.rand(3, 3, depth))
    return every_kernel

In [49]:
test_image = np.random.rand(10, 10, 4)

In [50]:
def convolve_one_step(input_image, kernel_used):
    intermediate_values = []
    dimensions = input_image.shape
    final_values = []
    for i in range(dimensions[0] - 2):
        for j in range(dimensions[1] - 2):
            for k in range(dimensions[2] ):
                slice_of_image = input_image[i : i + 3, j : j + 3, k]
                real_kernel = kernel_used[:, :, k]
                value = np.vdot(slice_of_image, real_kernel)
                intermediate_values.append(value)
                #pdb.set_trace()
                if len(intermediate_values) == dimensions[2]:
                    final_values.append(sum_dimension(intermediate_values))
                    intermediate_values.clear()
    return np.array(final_values)

In [51]:
def sum_dimension(list_of_three):
    return sum(list_of_three)

In [52]:
image_after_conv_2 = np.array(convolve_one_step(array_image, kernel_3d)).reshape(dimensions[0], dimensions[1], 1)

NameError: name 'dimensions' is not defined

In [53]:
def max_pooling(image_array):
    pooling_values = []
    for m in range(0, image_array.shape[0], 2):
        for n in range(0, image_array.shape[1], 2):
            for o in range(image_array.shape[2]):
                if n + 2 > image_array.shape[1]:
                    slice_of_pooling = image_array[m : m + 2, n - 1: n + 1 , o]
                else:
                    slice_of_pooling = image_array[m : m + 2, n : n + 2 , o]
                pooling_values.append(slice_of_pooling.max())
                #pdb.set_trace()
    return pooling_values

In [54]:
filtr = test_image[0 : 2, 0 : 2, 0]

In [55]:
after_maxpool = np.array(max_pooling(test_image)).reshape(5, 5, 4)

In [56]:
after_maxpool.shape

(5, 5, 4)

In [57]:
def ReLUv1(x):
    after_relu = []
    for r in x:
        if r > 0:
            after_relu.append(r)
        else:
            after_relu.append(0)
    return after_relu

In [58]:
def ReLUv2(x):
    return np.maximum(0, x)

In [59]:
last_pool = np.random.rand(2, 2, 1)

In [60]:
def fully_connected(after_last_max_pool, number_of_nodes):
    almost_values = []
    nodes_values = []
    flatten_array = after_last_max_pool.reshape(4)
    weights = np.random.rand(len(flatten_array), number_of_nodes)
    for x in range(weights.shape[1]):
        for y in range(weights.shape[0]):
            if y == weights.shape[0] - 1:
                almost_values.append(weights[y : y + 1, x] * flatten_array[y])
                nodes_values.append(sum(almost_values))
                almost_values.clear()
            else:
                almost_values.append(weights[y : y + 1, x] * flatten_array[y])
    return nodes_values

In [61]:
fully_connected(last_pool, 4)

[array([1.48537304]),
 array([0.77726137]),
 array([1.28790015]),
 array([1.24852192])]

In [62]:
test_image = np.random.rand(5, 5, 4)

In [95]:
test_kernel = np.random.rand(3, 3, 4)

In [63]:
def reshape_to_matrix_image(height, width, kernel_height, kernel_width, depth):
    number_of_patches = np.prod(reshape_dimensions_convolution(height, width, kernel_height, kernel_width, 1))
    each_patch_size = kernel_height * kernel_width * depth
    return (number_of_patches, each_patch_size)

In [64]:
reshape_to_matrix_image(7, 7, 3, 3, 4)

(25, 36)

In [173]:
def convolve_one_step_vectorized(imput_image, kernel_used):
    shape_of_image = imput_image.shape
    kernel_used_shape = kernel_used.shape
    dim_to_reshape = reshape_dimensions_convolution(shape_of_image[0], shape_of_image[1], kernel_used_shape[0], kernel_used_shape[1], 1)
    reshaped_image  = reshape_to_matrix_valid(imput_image, (3, 3), 1, 9, -1)
    reshaped_kernel = np.squeeze(reshape_to_matrix_valid(kernel_used, (3, 3), 1, -1, 1))
    vectorized_output = np.matmul(reshaped_image, reshaped_kernel)
    return vectorized_output.reshape(dim_to_reshape)

In [170]:
def reshape_to_matrix_valid(image, patch_size, stride, rA, rB):
    shape = (image.shape[0] - patch_size[0] + 1,
             image.shape[1] - patch_size[1] + 1,
             patch_size[0],
             patch_size[1],
             image.shape[2])
    strides = (image.strides[0],
               image.strides[1],
               image.strides[0],
               image.strides[1],
               image.strides[2])
    patches = as_strided(image, shape, strides)
    reshaped_patches = patches.reshape(rA, rB)
    return reshaped_patches

In [175]:
convolve_one_step_vectorized(test_image, test_kernel)

array([[8.87827426, 8.00065074, 9.00165848],
       [6.86961361, 7.73265371, 8.82710737],
       [6.73487624, 8.26072762, 8.43180012]])

In [140]:
test_image.shape[0]

5

In [157]:
reshape_dimensions_convolution(5, 5, 3, 3, 1)

(3, 3)